In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 2
probp = 0.75
expon = 2 * alphap - 1
slope = 0.77872

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
def pickindex(frag_prob, frag_prob_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0, frag_prob_sum)
    sum_ = 0
    for index in range(len(frag_prob)):
        sum_ += frag_prob[index]
        if sum_ < r:
            continue
        else:
            return index

In [6]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [7]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [8]:
def realization_value(total_iteration, min_iteration, iteration_step):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    frag_prob_sum = 1.0 # normalization const

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    N_realization = []
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        index = pickindex(frag_prob, frag_prob_sum)
        
        if flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            frag_prob_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
    
    conserved_quantity = np.array(M_realization)
    
    return conserved_quantity

In [9]:
def ensemble_average(total_iteration, min_iteration, iteration_step, ensemble_size):

    data_points = int ((total_iteration - min_iteration)/iteration_step + 1)
    M_ensemble = np.zeros(data_points)
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, min_iteration, iteration_step)
        if i % 1000 == 0:
            print("working with realization ",i)
        if i in [20_000,40_000,60_000,80_000]:
            print(M_list)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    return M_average

In [10]:
import time
t1 = time.time()
realization_value(100_000, 10_000, 10_000)
t2 = time.time()
run_time = t2 - t1
print("run_time is ", run_time, " sec")

run_time is  0.5962059497833252  sec


In [11]:
ensemble_average(100_000, 10_000, 10_000, 100_000)

working with realization  0
working with realization  1000
working with realization  2000
working with realization  3000
working with realization  4000
working with realization  5000
working with realization  6000
working with realization  7000
working with realization  8000
working with realization  9000
working with realization  10000
working with realization  11000
working with realization  12000
working with realization  13000
working with realization  14000
working with realization  15000
working with realization  16000
working with realization  17000
working with realization  18000
working with realization  19000
working with realization  20000
[1.71499875 1.74867454 1.72645101 1.73356202 1.73476455 1.74108788
 1.73560638 1.73929815 1.72283332 1.72745457]
working with realization  21000
working with realization  22000
working with realization  23000
working with realization  24000
working with realization  25000
working with realization  26000
working with realization  27000
work

array([0.99902435, 0.99908795, 0.9990547 , 0.99903829, 0.99906526,
       0.99910908, 0.99909997, 0.99910946, 0.99909509, 0.99906448])